<a href="https://colab.research.google.com/github/SJButla/IMLOO/blob/main/IMLO_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Define transformations for the training set
my_train_transform = transforms.Compose([
    transforms.RandomResizedCrop(192),  # Randomly crop and resize the image to 192x192
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(30),  # Randomly rotate the image by up to 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change brightness, contrast, saturation, and hue
    transforms.ToTensor(),  # Convert image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image with mean and std deviation
])

# Define transformations for the test set
my_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize the image to 224x224
    transforms.CenterCrop(192),  # Center crop the image to 192x192
    transforms.ToTensor(),  # Convert image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image with mean and std deviation
])
# Load the Flowers102 dataset for training and test sets
flowers102_dataset = datasets.Flowers102(root='data', split='train', download=True, transform=my_train_transform)
test_data = datasets.Flowers102(root='data', split='test', download=True, transform=my_test_transform)
batch_size = 64  # Batch size for data loading
# Split the training dataset into training and validation sets (80% training, 20% validation)
dataset_size = len(flowers102_dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_data, val_data = random_split(flowers102_dataset, [train_size, val_size])

# Create DataLoader for training, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)


In [ ]:

# Load the Flowers102 dataset for training and test sets
flowers102_dataset = datasets.Flowers102(root='data', split='train', download=True, transform=my_train_transform)
test_data = datasets.Flowers102(root='data', split='test', download=True, transform=my_test_transform)
batch_size = 64  # Batch size for data loading
# Split the training dataset into training and validation sets (80% training, 20% validation)
dataset_size = len(flowers102_dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_data, val_data = random_split(flowers102_dataset, [train_size, val_size])

# Create DataLoader for training, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)


In [ ]:

# Define the neural network model
class FlowerClassifier(nn.Module):
    def __init__(self, dropout_r=0.5):
        super(FlowerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # First convolutional layer
        self.bn1 = nn.BatchNorm2d(64)  # Batch normalization for the first layer
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Second convolutional layer
        self.bn2 = nn.BatchNorm2d(128)  # Batch normalization for the second layer
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # Third convolutional layer
        self.bn3 = nn.BatchNorm2d(256)  # Batch normalization for the third layer
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)  # Fourth convolutional layer
        self.bn4 = nn.BatchNorm2d(512)  # Batch normalization for the fourth layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling layer
        self.dropout = nn.Dropout(dropout_r)  # Dropout layer for regularization
        self.fc1 = nn.Linear(512 * 12 * 12, 102)  # Fully connected layer for classification

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Apply conv1, batch norm, ReLU, and max pooling
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Apply conv2, batch norm, ReLU, and max pooling
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Apply conv3, batch norm, ReLU, and max pooling
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # Apply conv4, batch norm, ReLU, and max pooling
        x = x.view(-1, 512 * 12 * 12)  # Flatten the tensor for the fully connected layer
        x = self.dropout(x)  # Apply dropout
        x = self.fc1(x)  # Apply the fully connected layer
        return x


In [ ]:
# Hyperparameters
learning_r = 0.001  # Learning rate for the optimizer
weight_decay = 0.05  # Weight decay for regularization
dropout_r = 0.5  # Dropout rate for the dropout layer


# Set the device for computation (GPU if available, else CPU)
my_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize the model, loss function, and optimizer
model = FlowerClassifier(dropout_r=dropout_r).to(my_device)
loss_function = nn.CrossEntropyLoss()  # Loss function for classification
optimiser = optim.Adam(model.parameters(), lr=learning_r, weight_decay=weight_decay)  # Adam optimizer

# Setup the learning rate scheduler
step = optim.lr_scheduler.StepLR(optimiser, step_size=30, gamma=0.8)  # StepLR scheduler to reduce learning rate


In [ ]:
# Function to calculate accuracy
def calculate_accuracy(loader, model):
    model.eval()  # Set the model to evaluation mode
    correct = 0  # Initialize the count of correct predictions
    total = 0  # Initialize the count of total samples
    with torch.no_grad():  # Disable gradient computation
        for input, label in loader:  # Iterate over the data loader
            input = input.to(my_device)  # Move inputs to the device
            label = label.to(my_device)  # Move labels to the device
            outputs = model(input)  # Get model predictions
            _, preds = torch.max(outputs, 1)  # Get the predicted class
            correct += torch.sum(preds == label.data)  # Count the correct predictions
            total += label.size(0)  # Count the total samples
    return (correct.double() / total) * 100  # Return the accuracy as a percentage

# Function to train the model
def train_model(model, train_loader, val_loader, test_loader, loss_function, optimizer, scheduler, num_epochs=500):
    my_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_val_accuracy = 0.0  # Initialize the best validation accuracy
    for epoch in range(num_epochs):  # Iterate over the epochs
        model.train()  # Set the model to training mode
        running_loss = 0.0  # Initialize running loss
        correct = 0  # Initialize the count of correct predictions
        total = 0  # Initialize the count of total samples
        for input, label in train_loader:  # Iterate over the training data loader
            input = input.to(my_device)  # Move inputs to the device
            label = label.to(my_device)  # Move labels to the device
            optimiser.zero_grad()  # Zero the gradients
            outputs = model(input)  # Get model predictions
            loss = loss_function(outputs, label)  # Compute the loss
            loss.backward()  # Backpropagate the loss
            optimiser.step()  # Update the model parameters
            running_loss += loss.item()  # Accumulate the loss
            _, preds = torch.max(outputs, 1)  # Get the predicted class
            correct += torch.sum(preds == label.data)  # Count the correct predictions
            total += label.size(0)  # Count the total samples

        step.step()  # Update the learning rate after each epoch

        if (epoch + 1) % 50 == 0:  # Print information every 50 epochs
            epoch_loss = running_loss / len(train_loader.dataset)  # Compute the epoch loss
            train_accuracy = (correct.double() / total) * 100  # Compute the training accuracy
            val_accuracy = calculate_accuracy(val_loader, model)  # Compute the validation accuracy

            if val_accuracy > best_val_accuracy:  # Check if validation accuracy improved
                best_val_accuracy = val_accuracy  # Update the best validation accuracy
                torch.save(model.state_dict(), 'best_model.pth')  # Save the best model

            print(f'Epoch {epoch+1}/{num_epochs}, Batch Size: {batch_size}, Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%')

    print('Training complete')
    model.load_state_dict(torch.load('best_model.pth'))  # Load the best model
    return model

In [ ]:

# Train the model
trained_model = train_model(model, train_loader, val_loader, test_loader, loss_function, optimiser, step, num_epochs=500)

# Evaluate the model on the test set
def evaluate_model(model, test_loader):
    test_accuracy = calculate_accuracy(test_loader, model)
    print(f'Test Accuracy: {test_accuracy:.2f}%')

evaluate_model(trained_model, test_loader)